# Tarefa 4

<b>Alunos:</b> Andreza (164213), Gil (225323) e Yan (118982)

In [7]:
import numpy as np
import random
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from hyperopt import tpe, fmin, hp, STATUS_OK
from pyswarm import pso
from simanneal import Annealer

## Hiperparâmetros:
* C: 2^{-5} a 2^{15} (uniforme nos expoentes)
* gamma: 2^{-15} a 2^{3} (uniforme nos expoentes)
* epsilon: 0.05 a 1.0 (uniforme no intervalo)

## Carregamento dos dados

In [8]:
X = np.load("X.npy")
y = np.load("y.npy")

## Random Search

In [9]:
C_values = [2**float(expo) for expo in np.random.uniform(low = -5, high = 15, size = 125)]
gamma_values = [2**float(expo) for expo in np.random.uniform(low = -15, high = 3, size = 125)]
param_distributions = {'C': C_values, 
                       'gamma': gamma_values ,
                       'epsilon': np.random.uniform(low = 0.05, high = 1.0, size = 125)}

# Realiza a busca aleatória
# Se cv = None, utiliza 5 fold cross validation
random_search = RandomizedSearchCV(
    SVR(),
    param_distributions,
    random_state = 0,
    n_iter=125,
    scoring = 'neg_mean_squared_error',
    cv = None,
    refit = True,
    verbose=2)
random_search.fit(X, y)

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] END C=18253.10201373355, epsilon=0.5780399480725146, gamma=0.00019867095484023384; total time=   3.8s
[CV] END C=18253.10201373355, epsilon=0.5780399480725146, gamma=0.00019867095484023384; total time=   3.1s
[CV] END C=18253.10201373355, epsilon=0.5780399480725146, gamma=0.00019867095484023384; total time=   2.7s
[CV] END C=18253.10201373355, epsilon=0.5780399480725146, gamma=0.00019867095484023384; total time=   2.4s
[CV] END C=18253.10201373355, epsilon=0.5780399480725146, gamma=0.00019867095484023384; total time=   3.8s
[CV] END C=11815.929590264372, epsilon=0.5989392554900013, gamma=1.003361924084318; total time=   0.0s
[CV] END C=11815.929590264372, epsilon=0.5989392554900013, gamma=1.003361924084318; total time=   0.0s
[CV] END C=11815.929590264372, epsilon=0.5989392554900013, gamma=1.003361924084318; total time=   0.0s
[CV] END C=11815.929590264372, epsilon=0.5989392554900013, gamma=1.003361924084318; total tim

[CV] END C=0.18209645426015236, epsilon=0.5382489576574371, gamma=0.01011105094744343; total time=   0.0s
[CV] END C=0.18209645426015236, epsilon=0.5382489576574371, gamma=0.01011105094744343; total time=   0.0s
[CV] END C=0.18209645426015236, epsilon=0.5382489576574371, gamma=0.01011105094744343; total time=   0.0s
[CV] END C=1716.6363133126672, epsilon=0.6531289882019802, gamma=0.00035217789816756486; total time=   0.3s
[CV] END C=1716.6363133126672, epsilon=0.6531289882019802, gamma=0.00035217789816756486; total time=   0.4s
[CV] END C=1716.6363133126672, epsilon=0.6531289882019802, gamma=0.00035217789816756486; total time=   0.7s
[CV] END C=1716.6363133126672, epsilon=0.6531289882019802, gamma=0.00035217789816756486; total time=   0.4s
[CV] END C=1716.6363133126672, epsilon=0.6531289882019802, gamma=0.00035217789816756486; total time=   0.4s
[CV] END C=0.03334987059941054, epsilon=0.8537369044298997, gamma=1.862761031646447; total time=   0.0s
[CV] END C=0.03334987059941054, epsilo

[CV] END C=0.03334987059941054, epsilon=0.10625311030913044, gamma=0.01984998563675404; total time=   0.0s
[CV] END C=0.03334987059941054, epsilon=0.10625311030913044, gamma=0.01984998563675404; total time=   0.0s
[CV] END C=0.03334987059941054, epsilon=0.10625311030913044, gamma=0.01984998563675404; total time=   0.0s
[CV] END C=847.9989850213742, epsilon=0.5948122274026175, gamma=5.022182348331604e-05; total time=   0.0s
[CV] END C=847.9989850213742, epsilon=0.5948122274026175, gamma=5.022182348331604e-05; total time=   0.0s
[CV] END C=847.9989850213742, epsilon=0.5948122274026175, gamma=5.022182348331604e-05; total time=   0.0s
[CV] END C=847.9989850213742, epsilon=0.5948122274026175, gamma=5.022182348331604e-05; total time=   0.0s
[CV] END C=847.9989850213742, epsilon=0.5948122274026175, gamma=5.022182348331604e-05; total time=   0.0s
[CV] END C=97.62198200710652, epsilon=0.48602292209985554, gamma=0.4965974618191522; total time=   0.0s
[CV] END C=97.62198200710652, epsilon=0.48602

[CV] END C=1941.9609697319565, epsilon=0.16623166283626506, gamma=0.00035217789816756486; total time=   0.5s
[CV] END C=1941.9609697319565, epsilon=0.16623166283626506, gamma=0.00035217789816756486; total time=   0.6s
[CV] END C=1941.9609697319565, epsilon=0.16623166283626506, gamma=0.00035217789816756486; total time=   0.7s
[CV] END C=1941.9609697319565, epsilon=0.16623166283626506, gamma=0.00035217789816756486; total time=   0.7s
[CV] END C=1941.9609697319565, epsilon=0.16623166283626506, gamma=0.00035217789816756486; total time=   0.9s
[CV] END C=12789.923318680116, epsilon=0.861586039231038, gamma=0.015245556183256034; total time=   0.0s
[CV] END C=12789.923318680116, epsilon=0.861586039231038, gamma=0.015245556183256034; total time=   0.0s
[CV] END C=12789.923318680116, epsilon=0.861586039231038, gamma=0.015245556183256034; total time=   0.0s
[CV] END C=12789.923318680116, epsilon=0.861586039231038, gamma=0.015245556183256034; total time=   0.0s
[CV] END C=12789.923318680116, epsi

[CV] END C=470.87525944055, epsilon=0.23061537167578067, gamma=7.981008449648598e-05; total time=   0.0s
[CV] END C=470.87525944055, epsilon=0.23061537167578067, gamma=7.981008449648598e-05; total time=   0.0s
[CV] END C=3230.981123660203, epsilon=0.6485969936832545, gamma=0.0002095424747774296; total time=   0.9s
[CV] END C=3230.981123660203, epsilon=0.6485969936832545, gamma=0.0002095424747774296; total time=   0.6s
[CV] END C=3230.981123660203, epsilon=0.6485969936832545, gamma=0.0002095424747774296; total time=   0.7s
[CV] END C=3230.981123660203, epsilon=0.6485969936832545, gamma=0.0002095424747774296; total time=   0.5s
[CV] END C=3230.981123660203, epsilon=0.6485969936832545, gamma=0.0002095424747774296; total time=   0.7s
[CV] END C=6.130141410584192, epsilon=0.6042660216256781, gamma=0.02227717785778305; total time=   0.0s
[CV] END C=6.130141410584192, epsilon=0.6042660216256781, gamma=0.02227717785778305; total time=   0.0s
[CV] END C=6.130141410584192, epsilon=0.604266021625

[CV] END C=3.0200652225667453, epsilon=0.37350305752629104, gamma=0.0009038530708606632; total time=   0.0s
[CV] END C=3.0200652225667453, epsilon=0.37350305752629104, gamma=0.0009038530708606632; total time=   0.0s
[CV] END C=3.0200652225667453, epsilon=0.37350305752629104, gamma=0.0009038530708606632; total time=   0.0s
[CV] END C=3.0200652225667453, epsilon=0.37350305752629104, gamma=0.0009038530708606632; total time=   0.0s
[CV] END C=3.0200652225667453, epsilon=0.37350305752629104, gamma=0.0009038530708606632; total time=   0.0s
[CV] END C=596.0562418006148, epsilon=0.27272879017690466, gamma=0.020677953765508246; total time=   0.0s
[CV] END C=596.0562418006148, epsilon=0.27272879017690466, gamma=0.020677953765508246; total time=   0.0s
[CV] END C=596.0562418006148, epsilon=0.27272879017690466, gamma=0.020677953765508246; total time=   0.0s
[CV] END C=596.0562418006148, epsilon=0.27272879017690466, gamma=0.020677953765508246; total time=   0.0s
[CV] END C=596.0562418006148, epsilo

[CV] END C=752.6440406799057, epsilon=0.5780399480725146, gamma=0.0017296995991167648; total time=   0.1s
[CV] END C=752.6440406799057, epsilon=0.5780399480725146, gamma=0.0017296995991167648; total time=   0.0s
[CV] END C=752.6440406799057, epsilon=0.5780399480725146, gamma=0.0017296995991167648; total time=   0.1s
[CV] END C=2.608214820199381, epsilon=0.14185645123310087, gamma=0.0017296995991167648; total time=   0.0s
[CV] END C=2.608214820199381, epsilon=0.14185645123310087, gamma=0.0017296995991167648; total time=   0.0s
[CV] END C=2.608214820199381, epsilon=0.14185645123310087, gamma=0.0017296995991167648; total time=   0.0s
[CV] END C=2.608214820199381, epsilon=0.14185645123310087, gamma=0.0017296995991167648; total time=   0.0s
[CV] END C=2.608214820199381, epsilon=0.14185645123310087, gamma=0.0017296995991167648; total time=   0.0s
[CV] END C=110.95170236098846, epsilon=0.2556693498802824, gamma=0.048637435329376026; total time=   0.0s
[CV] END C=110.95170236098846, epsilon=0.

[CV] END C=14468.428154317606, epsilon=0.9326973389634642, gamma=0.0006387045688201382; total time=   1.7s
[CV] END C=14468.428154317606, epsilon=0.9326973389634642, gamma=0.0006387045688201382; total time=   1.2s
[CV] END C=14468.428154317606, epsilon=0.9326973389634642, gamma=0.0006387045688201382; total time=   1.9s
[CV] END C=14468.428154317606, epsilon=0.9326973389634642, gamma=0.0006387045688201382; total time=   1.3s
[CV] END C=14468.428154317606, epsilon=0.9326973389634642, gamma=0.0006387045688201382; total time=   1.9s
[CV] END C=161.23411436439807, epsilon=0.3048313311389562, gamma=1.6912107709335567; total time=   0.0s
[CV] END C=161.23411436439807, epsilon=0.3048313311389562, gamma=1.6912107709335567; total time=   0.0s
[CV] END C=161.23411436439807, epsilon=0.3048313311389562, gamma=1.6912107709335567; total time=   0.0s
[CV] END C=161.23411436439807, epsilon=0.3048313311389562, gamma=1.6912107709335567; total time=   0.0s
[CV] END C=161.23411436439807, epsilon=0.30483133

RandomizedSearchCV(estimator=SVR(), n_iter=125,
                   param_distributions={'C': [10761.260343945227,
                                              554.2332287963202,
                                              1024.366170848103,
                                              0.858234440407419,
                                              499.80518225336533,
                                              10.730856640794084,
                                              12.979052453100051,
                                              0.4934384541144724,
                                              474.920386446124,
                                              0.06836242046295685,
                                              0.9676443324327539,
                                              2996.6357111056323,
                                              0.2078739673615182,
                                              1631.8803151267098,
                                

In [10]:
print("Melhores parâmetros: ", random_search.best_params_)
regressor = SVR(kernel = 'rbf', C = random_search.best_params_['C'],
                gamma = random_search.best_params_['gamma'],
                epsilon = random_search.best_params_['epsilon'])

scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Melhores parâmetros:  {'gamma': 3.114511279603333e-05, 'epsilon': 0.8783886350999086, 'C': 10840.666926569591}
RMSE: 3.9148759393643515


## Grid Search

Fazemos um grid search com grid de 5x5x5 selecionando uniformemente 5 expoentes de C e gamma e 5 valores de epsilon

In [11]:
C_values = [2**float(expo) for expo in np.random.uniform(low = -5, high = 15, size = 5)]
gamma_values = [2**float(expo) for expo in np.random.uniform(low = -15, high = 3, size = 5)]
param_grid = {'C': C_values, 
              'gamma': gamma_values ,
              'epsilon': np.linspace(start = 0.05, stop = 1.0, num = 5)}
print(param_grid)

# Realiza o Grid Search utilizando os parametros definidos em param_grid
grid_search = GridSearchCV(SVR(),
                           param_grid,
                           cv = None,
                           scoring = 'neg_mean_squared_error',
                           refit = True,
                           verbose = 2)
grid_search.fit(X, y)

{'C': [87.82088740733852, 0.034214537725834035, 14.959166778918055, 6.978433942002368, 0.04076975908020163], 'gamma': [0.3871971278235247, 0.0004865167139595476, 0.0019246735621156936, 0.00011560326003805887, 0.09886741617577692], 'epsilon': array([0.05  , 0.2875, 0.525 , 0.7625, 1.    ])}
Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] END C=87.82088740733852, epsilon=0.05, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.05, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.05, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.05, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.05, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.05, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.05, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=87.820

[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=87.82088740733852, epsilon=0.7625, gamma=0.09886

[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=0.034214537725834035, epsilon=0.525, g

[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=0.05, gamma=0.09886741617577692;

[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0004865167139595476; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=14.959166778918055, epsilon=1.0, gamma=0.0019246735621156936; total time=   0.

[CV] END C=6.978433942002368, epsilon=0.525, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.525, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.525, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.525, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.525, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.525, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.525, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.7625, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.7625, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.7625, gamma=0.3871971278235247; total time=   0.0s
[CV] END C=6.978433942002368, epsilon=0.7625, gamma=0.3871971278235247; total time= 

[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.0019246735621156936; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.00011560326003805887; total time=   0.0s
[CV] END C=0.04076975908020163, epsilon=0.2875, gamma=0.09886741617577692; total time=   0.0s
[CV] END C=0.04076975908020163, epsil

GridSearchCV(estimator=SVR(),
             param_grid={'C': [87.82088740733852, 0.034214537725834035,
                               14.959166778918055, 6.978433942002368,
                               0.04076975908020163],
                         'epsilon': array([0.05  , 0.2875, 0.525 , 0.7625, 1.    ]),
                         'gamma': [0.3871971278235247, 0.0004865167139595476,
                                   0.0019246735621156936,
                                   0.00011560326003805887,
                                   0.09886741617577692]},
             scoring='neg_mean_squared_error', verbose=2)

In [12]:
print("Melhores parâmetros: ", grid_search.best_params_)
regressor = SVR(kernel = 'rbf', C = grid_search.best_params_['C'],
                gamma = grid_search.best_params_['gamma'],
                epsilon = grid_search.best_params_['epsilon'])

scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Melhores parâmetros:  {'C': 87.82088740733852, 'epsilon': 1.0, 'gamma': 0.0004865167139595476}
RMSE: 5.191932339666912


## Bayesian Search

A otimização bayesiana utilizando a bibliotec hyperopt requer a definição de uma função objetivo 
a ser minimizada. Como queremos maximizar a accuracia, utilizamos loss = -acc para ser minimizada. 

In [13]:
def objective_func(search_space):
    C =  search_space['C']
    gamma = search_space['gamma']
    epsilon = search_space['epsilon']
    regressor = SVR(**{'C': 2**C, 'gamma': 2**gamma, 'epsilon': epsilon})
    
    acc = cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=None).mean()
    return {'loss': -acc, 'status': STATUS_OK}

In [14]:
bayesian_search_space = {
    'C': hp.uniform('C', -5, 15),
    'gamma': hp.uniform('gamma', -15, 3),
    'epsilon': hp.uniform('epsilon', 0.05, 1.0)   
}

best_params = fmin(objective_func, bayesian_search_space, algo=tpe.suggest, max_evals=125)
best_parameters = {
    'C': 2**best_params['C'],
    'gamma': 2**best_params['gamma'],
    'epsilon': best_params['epsilon']
}

100%|██████████| 125/125 [07:24<00:00,  3.56s/trial, best loss: 14.772166181490135]


In [15]:
print("Melhores parâmetros:", best_parameters)
regressor = SVR(C=best_parameters['C'], gamma=best_parameters['gamma'], epsilon=best_parameters['epsilon'])
scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Melhores parâmetros: {'C': 24035.379966761484, 'gamma': 3.062443350545391e-05, 'epsilon': 0.4450786080248112}
RMSE: 3.8434575815911036


## PSO

In [ ]:
def funcao(param):  
    svr = SVR(kernel='rbf', C=param[0], gamma=param[1], epsilon=param[2])
    rmse = make_scorer(mean_squared_error, squared=False, greater_is_better=False)
    scores = cross_val_score(svr, X, y, cv=5, scoring=rmse)#'neg_root_mean_squared_error')
    rmse_mean = -np.mean(scores)
    return rmse_mean

lb = np.array([2**(-5),2**(-15),0.05])
ub = np.array([2**15,2**3,1.0])

xopt, fopt = pso(funcao, lb, ub, swarmsize=11, maxiter=11)

C_opt = str(xopt[0])
gamma_opt = str(xopt[1])
epsilon_opt = str(xopt[2])

print("C ótimo: " + C_opt)
print("gamma ótimo: " + gamma_opt)
print("epsilon ótimo: " + epsilon_opt)
print("RMSE: " + str(fopt))

## Simulated annealing

In [18]:
class SimAnn(Annealer):

    def move(self):
        self.state[0] = random.uniform(2**exp_min_C, 2**exp_max_C)
        self.state[1] = random.uniform(2**exp_min_gamma, 2**exp_max_gamma)
        self.state[2] = random.uniform(min_epsilon, max_epsilon)
    
    def energy(self):
        svr = SVR(kernel='rbf', C=self.state[0], gamma=self.state[1], epsilon=self.state[2])
        rmse = make_scorer(mean_squared_error, squared=False, greater_is_better=False)
        scores = cross_val_score(svr, X, y, cv=5, scoring=rmse)#'neg_root_mean_squared_error')
        rmse_mean = -np.mean(scores)
        
        return rmse_mean

exp_min_C = -5
exp_max_C = 15
exp_min_gamma = -15
exp_max_gamma = 3
min_epsilon = 0.05
max_epsilon = 1.0
C = random.uniform(2**exp_min_C, 2**exp_max_C)
gamma = random.uniform(2**exp_min_gamma, 2**exp_max_gamma)
epsilon = random.uniform(min_epsilon, max_epsilon)
init_state = [C, gamma, epsilon]
sa = SimAnn(init_state)
sa.steps = 125
best_params, rmse = sa.anneal()
print('C ótimo: ' + str(best_params[0]))
print('gamma ótimo: ' + str(best_params[1]))
print('epsilon ótimo: ' + str(best_params[2]))
print('RMSE: ' + str(rmse))

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          9.15   100.00%     0.00%     0:00:37     0:00:00

C ótimo: 19216.741891201225
gamma ótimo: 0.0007891170836278364
epsilon ótimo: 0.468709376054373
RMSE: 7.077409001227068
